In [ ]:
import sys
import argparse
import numpy as np
import scipy.signal as sp
import scipy.interpolate
import processing as pr
import matplotlib.pyplot as plt
import pandas as pd
from ruamel.yaml import YAML as ym
import datetime
import copy

%matplotlib widget
import mplcursors

sys.path.append("../preprocessing")
from generate_chirp import generate_chirp

# Widgets are only needed if you want to use the interactive plot at the end
# Installation instructions: https://ipywidgets.readthedocs.io/en/latest/user_install.html
import ipywidgets as widgets

import scipy.fft

In [ ]:
# Just an example for pulse measurements
prefix = "../../drone/radar_data/20220321-tellbreen/20220321_052716"

zero_sample_idx = 159

yaml_file = prefix + "_config.yaml"
bin_file = prefix + "_rx_samps.bin"

# Initialize Constants
yaml = ym()
with open(yaml_file) as stream:
    config = yaml.load(stream)
    sample_rate = config["PLOT"]["sample_rate"]    # Hertz
    sig_speed = config["PLOT"]["sig_speed"] / np.sqrt(3.17)

    #expected_n_rxs = int(config['CHIRP']['num_pulses'] / config['CHIRP']['num_presums'])
    
    rx_len_samples = int(config['CHIRP']['rx_duration'] * config['GENERATE']['sample_rate'])
    
rx_samps = bin_file

config_modified = copy.deepcopy(config)
config_modified['GENERATE']['window'] = 'rectangular'
config_modified['GENERATE']['chirp_length'] = 300e-6

In [ ]:
config_modified['GENERATE']['chirp_bandwidth']

In [ ]:
config_modified['RF0']['freq'] + (config_modified['GENERATE']['chirp_bandwidth']/2)

In [ ]:
# Read and plot RX/TX
# This cell loads all of the data - it can take a while with a large file. You don't need to re-run this cell if you only change n_stack
_, tx_sig = generate_chirp(config_modified) # TODO
start_freq = config_modified['RF0']['freq'] - (config_modified['GENERATE']['chirp_bandwidth']/2)
end_freq = config_modified['RF0']['freq'] + (config_modified['GENERATE']['chirp_bandwidth']/2)

pr.plotChirpVsTime(tx_sig, 'Transmitted Chirp', sample_rate)
print(f"len(tx_sig): {len(tx_sig)}")

t0 = datetime.datetime.now()
rx_sig = pr.extractSig(rx_samps)
print(datetime.datetime.now() - t0)

In [ ]:
""" Load S11 data from FieldFox CSV files """
def load_fieldfox_s1p_data(filename):
    return pd.read_csv(filename, header=1, names=["frequency", "magnitude", "angle"], comment="!", sep='\t')

#s1p = load_fieldfox_s1p_data("/home/thomas/Documents/StanfordGrad/RadioGlaciology/antennas/20220303-rev2p5-antenna-checkout/03.Mar.2022.13.07_sn10_lab.s1p")
s1p = load_fieldfox_s1p_data("/home/thomas/Documents/StanfordGrad/RadioGlaciology/antennas/20220305-aeromao-installation-inside/05.Mar.2022.13.42_SN13_WING.s1p")

s11_derived_transmit_weights = 1 - 10**(s1p['magnitude'] / 20) # approximate portion of signal transmitted
transmit_weight_interpolator = scipy.interpolate.interp1d(s1p['frequency'], s11_derived_transmit_weights, kind='linear')
transmit_phase_interpolator = scipy.interpolate.interp1d(s1p['frequency'], s1p['angle'] * (np.pi/180), kind='linear')

inst_chirp_freq = np.linspace(start_freq, end_freq, len(tx_sig))
tx_sig_antenna_filtered = tx_sig * (transmit_weight_interpolator(inst_chirp_freq)**2) * np.exp(1j * transmit_phase_interpolator(inst_chirp_freq))

fig, ax = pr.plotChirpVsTime(tx_sig_antenna_filtered, 'Transmitted Chirp w/ Simulated Antenna', sample_rate)
fig.show()

#pd.read_csv("/home/thomas/Documents/StanfordGrad/RadioGlaciology/antennas/20220303-rev2p5-antenna-checkout/03.Mar.2022.13.07_sn10_lab.s1p", header=1, names=["frequency", "magnitude", "angle"], comment="!", sep='\t')

In [ ]:
def create_single_synthetic_rx_signal(tx_sig, rx_len_samples, delays={zero_sample_idx: -4}, phase_noise_std=0.0, noise_power_db=0.0):
    rx_sig = np.zeros(rx_len_samples, dtype=np.complex64)
    
    # Delayed copies
    for delay_samples, magnitude_db in delays.items():
        magnitude = 10 ** (magnitude_db / 20)
        len_delayed_chirp = np.minimum(len(tx_sig), len(rx_sig)-delay_samples)
        rx_sig[delay_samples:delay_samples+len_delayed_chirp] += magnitude * tx_sig[:len_delayed_chirp]
    
    # Phase noise
    rx_sig *= np.exp(1j*phase_noise_std*np.random.randn())
    
    # White noise
    noise_amplitude = 10**(noise_power_db/20)
    noise_mag = noise_amplitude * np.random.randn(len(rx_sig))
    noise_phase = np.random.uniform(size=len(rx_sig), low=0,high=2*np.pi)
    noise = noise_mag * np.exp(1j*noise_phase)
    rx_sig += noise
    
    return rx_sig

rx_sig = create_single_synthetic_rx_signal(tx_sig, int(len(tx_sig)*1.5),
                                           delays={zero_sample_idx: -3, zero_sample_idx+40: -30},
                                           phase_noise_std=0.0007527001552626,
                                           noise_power_db=-3
                                          )

fig, axs = pr.plotChirpVsTime(rx_sig, 'Recieved Chirp', sample_rate)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5), facecolor='white')


for n_stack in [1000]:

    rx_sigs = [create_single_synthetic_rx_signal(tx_sig_antenna_filtered, int(len(tx_sig)*1.5),
                                                  delays={zero_sample_idx: -3, zero_sample_idx+40: -30},
                                                  phase_noise_std=0,
                                                  noise_power_db=-10
                                                ) for n in range(n_stack)]
    rx_sigs = np.array(rx_sigs)
    print(np.shape(rx_sigs))
    rx_sig = np.mean(rx_sigs, axis=0)

    xcorr_res_tmp = sp.correlate(rx_sig, tx_sig, mode='valid', method='direct')

    distance_to_reflector = np.linspace(0, np.shape(xcorr_res_tmp)[0]/sample_rate, np.shape(xcorr_res_tmp)[0]) * sig_speed / 2
    distance_to_reflector = distance_to_reflector - distance_to_reflector[zero_sample_idx]

    line = ax.plot(distance_to_reflector, 20*np.log10(np.abs(xcorr_res_tmp)), label=f"n_stack = {n_stack}")
    mplcursors.cursor(line)

#ax.set_ylim(-20,40)
ax.set_xlim(-100, 200)
ax.set_xlabel('Distance [m]')
ax.set_ylabel('Power [dB]')
ax.grid()
fig.show()

In [ ]:
def butter_lowpass(cutoff, fs, order=5):
    return scipy.signal.butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = scipy.signal.lfilter(b, a, data)
    return y

# fig, ax = plt.subplots()
# ax.plot(chirp_scaling)
# fig.show()

In [ ]:
# def butter_lowpass(cutoff, fs, order=5):
#     return scipy.signal.butter(order, cutoff, fs=fs, btype='low', analog=False)

# def butter_lowpass_filter(data, cutoff, fs, order=5):
#     b, a = butter_lowpass(cutoff, fs, order=order)
#     y = scipy.signal.lfilter(b, a, data)
#     return y

# chirp_scaling = np.cumsum(np.random.randn(len(tx_sig)))
# chirp_scaling = butter_lowpass_filter(chirp_scaling, 0.001, 1)
# chirp_scaling = chirp_scaling - np.min(chirp_scaling)
# chirp_scaling = chirp_scaling / np.max(chirp_scaling)

tx_sig_autocorr = sp.correlate(tx_sig, tx_sig, mode='same', method='auto')

distance_to_reflector_autocorr = np.linspace(0, len(tx_sig_autocorr)/sample_rate, len(tx_sig_autocorr)) * sig_speed / 2
distance_to_reflector_autocorr = distance_to_reflector_autocorr - distance_to_reflector_autocorr[len(tx_sig_autocorr)//2]

fig, ax = plt.subplots(figsize=(10,5))
tx_sig_autocorr_db = 20*np.log10(np.abs(tx_sig_autocorr))
ax.plot(distance_to_reflector_autocorr, tx_sig_autocorr_db - np.max(tx_sig_autocorr_db), label="TX sig autocorr without filtering", c='black')

for n in range(10):
    # Generate random scaling
    chirp_scaling = np.cumsum(np.random.randn(len(tx_sig)))
    chirp_scaling = butter_lowpass_filter(chirp_scaling, 0.01, 1)
    chirp_scaling = chirp_scaling - np.min(chirp_scaling)
    chirp_scaling = chirp_scaling / np.max(chirp_scaling)
    
    # Calculate correlation
    tx_sig_modified = sp.correlate(tx_sig, tx_sig * chirp_scaling, mode='same', method='auto')
    #tx_sig_modified = sp.correlate(tx_sig, tx_sig * chirp_scaling + 2*np.random.randn(len(tx_sig)), mode='same', method='auto')
    
    # Plot
    ax.plot(distance_to_reflector_autocorr, 20*np.log10(np.abs(tx_sig_modified)) - np.max(tx_sig_autocorr_db), alpha=0.2)

ax.set_xlim(-150,150)
#ax.set_xlim(len(tx_sig_autocorr)/2 - 200, len(tx_sig_autocorr)/2 + 200)
ax.set_ylim(-100,0)
ax.legend()